In [1]:
pip uninstall gymnasium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install package_name --user

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install gymnasium

  Using cached gymnasium-1.1.1-py3-none-any.whl (965 kB)
  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\numpy-1.19.2.dist-info\\direct_url.json'
Consider using the `--user` option or check the permissions.



In [1]:
import  gym
import numpy as np
from my_q_table import q_table, discretize_state  # or copy directly here

env = gym.make('MountainCar-v0', render_mode='human')
state, _ = env.reset()
done = False

while not done:
    s_disc = discretize_state(state)
    action = np.argmax(q_table[s_disc])
    state, reward, done, truncated, info = env.step(action)
    env.render()

env.close()

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


ModuleNotFoundError: No module named 'my_q_table'

In [2]:
import  gym
import numpy as np
import random

# --- Initialize Environment ---
env = gym.make("FrozenLake-v1", is_slippery=True)  # deterministic environment
n_states = env.observation_space.n
n_actions = env.action_space.n

# --- Hyperparameters ---
EPSILON = 0.3      # exploration rate
ALPHA = 0.3        # learning rate
GAMMA = 0.99       # discount factor
EPISODES = 10000

# --- Q-table Initialization ---
Q = np.zeros((n_states, n_actions))

# --- Choose Action ---
def choose_action(state):
    if random.uniform(0, 1) < EPSILON:
        return env.action_space.sample()  # Explore
    else:
        return np.argmax(Q[state, :])     # Exploit best known action

# --- Update Q-table ---
def update_q_table(state, action, reward, next_state, done):
    best_next = np.max(Q[next_state, :])
    Q[state, action] += ALPHA * (reward + GAMMA * best_next * (not done) - Q[state, action])


# --- Training Loop ---
def train_agent():
    rewards_per_episode = []
    for episode in range(EPISODES):
        state, _ = env.reset()
        done = False
        total_reward = 0

        while not done:
            action = choose_action(state)
            next_state, reward, done, truncated, info = env.step(action)
            update_q_table(state, action, reward, next_state, done)
            state = next_state

        rewards_per_episode.append(total_reward)

        # Progress print
        if episode % 500 == 0 or episode == 1:
            avg_reward = np.mean(rewards_per_episode[-100:])
            print(f"Episode {episode:5d} | Reward: {total_reward:.2f} | "
                  f"Avg(100): {avg_reward:.3f} | Epsilon: {EPSILON:.3f}")

    return Q, rewards_per_episode
        

# --- Evaluation Function (place AFTER training) ---
def evaluate_agent(episodes=100):
    total_rewards = 0
    for _ in range(episodes):
        state, _ = env.reset()
        done = False
        while not done:
            action = np.argmax(Q[state, :])  # Always take best known action
            state, reward, done, truncated, info = env.step(action)
            total_rewards += reward
    avg_reward = total_rewards / episodes
    print(f"\n Average reward over {episodes} evaluation episodes: {avg_reward:.2f}")
    return avg_reward

# --- Run Training ---
Q, reward_per_episode = train_agent()

# --- Show Learned Q-table ---
print("\nLearned Q-table after training:\n")
print(np.round(Q, 3))  # rounded for readability

# --- Evaluate the Agent ---
evaluate_agent()

C:\Users\Student\AppData\Roaming\Python\Python38\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
Episode     0 | Reward: 0.00 | Avg(100): 0.000 | Epsilon: 0.300
Episode     1 | Reward: 0.00 | Avg(100): 0.000 | Epsilon: 0.300
Episode   500 | Reward: 0.00 | Avg(100): 0.000 | Epsilon: 0.300
Episode  1000 | Reward: 0.00 | Avg(100): 0.000 | Epsilon: 0.300
Episode  1500 | Reward: 0.00 | Avg(100): 0.000 | Epsilon: 0.300
Episode  2000 | Reward: 0.00 | Avg(100): 0.000 | Epsilon: 0.300
Episode  2500 | Reward: 0.00 | Avg(100): 0.000 | Epsilon: 0.300
Episode  3000 | Reward: 0.00 | Avg(100): 0.000 | Epsilon: 0.300
Episode  3500 | Reward: 0.00 | Avg(100): 0.000 | Epsilon: 0.300
Episode  4000 | Reward: 0.00 | Avg(100): 0.000 | Epsilon: 0.300
Episode  4500 | Reward: 0.00 | Avg(100): 0.000 | Epsilon: 0.300
Episode  5000 | Reward: 0.00 | Avg(100): 0.000 | E

0.0

In [8]:
import gym
import numpy as np
import matplotlib.pyplot as plt

# --- 1. Create Environment ---
env = gym.make("MountainCar-v0", render_mode="human")

# --- 2. Discretize the state space ---
num_bins = (20, 20)  # (position, velocity)
obs_space_low = env.observation_space.low
obs_space_high = env.observation_space.high
obs_space_bins = [
    np.linspace(obs_space_low[i], obs_space_high[i], num_bins[i]) for i in range(len(num_bins))
]

def discretize_state(state):
    """Convert continuous state to discrete bin index tuple"""
    indices = []
    for i in range(len(state)):
        idx = np.digitize(state[i], obs_space_bins[i]) - 1
        idx = min(num_bins[i]-1, max(0, idx))  # clamp to valid range
        indices.append(idx)
    return tuple(indices)

# --- 3. Initialize Q-table ---
q_table = np.random.uniform(low=-1, high=1, size=(num_bins[0], num_bins[1], env.action_space.n))

# --- 4. Hyperparameters ---
alpha = 0.1        # Learning rate
gamma = 0.99       # Discount factor
epsilon = 1.0      # Exploration rate
epsilon_decay = 0.995
epsilon_min = 0.01
episodes = 5000

# --- 5. Training Loop ---
rewards_per_episode = []

for episode in range(episodes):
    state = discretize_state(env.reset()[0])
    total_reward = 0
    done = False

    while not done:
        # ε-greedy action
        if np.random.random() < epsilon:
            action = np.random.randint(env.action_space.n)
        else:
            action = np.argmax(q_table[state])

        next_obs, reward, done, truncated, info = env.step(action)
        next_state = discretize_state(next_obs)

        # Update Q-value
        best_next_q = np.max(q_table[next_state])
        current_q = q_table[state + (action,)]
        new_q = (1 - alpha) * current_q + alpha * (reward + gamma * best_next_q)
        q_table[state + (action,)] = new_q

        state = next_state
        total_reward += reward

    # Decay epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    rewards_per_episode.append(total_reward)

    # Optional: Print progress
    if (episode+1) % 500 == 0:
        avg_reward = np.mean(rewards_per_episode[-500:])
        print(f"Episode {episode+1}, avg reward (last 500): {avg_reward:.2f}")

# --- 6. Plot learning curve ---
plt.plot(rewards_per_episode)
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.title('Q-Learning on MountainCar-v0 with Bins')
plt.show()

# --- 7. Test the trained agent ---
test_episodes = 5
for ep in range(test_episodes):
    obs = env.reset()[0]
    state = discretize_state(obs)
    done = False
    total_reward = 0

    while not done:
        env.render()
        action = np.argmax(q_table[state])
        obs, reward, done, truncated, info = env.step(action)
        state = discretize_state(obs)
        total_reward += reward

    print(f"Test Episode {ep+1}: Total Reward = {total_reward}")
env.close()

C:\Users\Student\AppData\Roaming\Python\Python38\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


{0: array([0.1, 0. ]), 1: array([0.86482753, 0.        ])}


In [20]:
import numpy as np
import random

class Environment:
    def __init__(self):
        self.state = 0  # Initial state

    def step(self, action):
        if action == 0:
            reward = 1
            self.state = 1
        else:
            reward = -1
            self.state = 0
        return self.state, reward

class SARSA:
    def __init__(self, actions, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.actions = actions
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}

    def get_action(self, state):
        if state not in self.q_table:
            self.q_table[state] = np.zeros(len(self.actions))
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(self.actions)
        else:
            return self.actions[np.argmax(self.q_table[state])]

    def update(self, state, action, reward, next_state, next_action):
        if state not in self.q_table:
            self.q_table[state] = np.zeros(len(self.actions))
        if next_state not in self.q_table:
            self.q_table[next_state] = np.zeros(len(self.actions))
        
        target = reward + self.gamma * self.q_table[next_state][self.actions.index(next_action)]
        self.q_table[state][self.actions.index(action)] += self.alpha * (target - self.q_table[state][self.actions.index(action)])

# Initialize environment and agent
env = Environment()
actions = [0, 1]
sarsa_agent = SARSA(actions)

# Simulate an episode
state = env.state
action = sarsa_agent.get_action(state)
for step in range(10):  # Run for 10 steps
    next_state, reward = env.step(action)
    next_action = sarsa_agent.get_action(next_state)
    sarsa_agent.update(state, action, reward, next_state, next_action)
    
   

In [21]:
# Example usage
env = Environment()
actions = [0, 1]
sarsa_agent = SARSA(actions)


In [26]:
import numpy as np
import random

class Environment:
    def __init__(self):
        self.state = 0

    def step(self, action):
        if action == 0:
            reward = 1
            self.state = 1
        else:
            reward = -1
            self.state = 0
        return self.state, reward

class SARSA:
    def __init__(self, actions, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.actions = actions
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}

    def get_action(self, state):
        if state not in self.q_table:
            self.q_table[state] = np.zeros(len(self.actions))
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(self.actions)
        else:
            return self.actions[np.argmax(self.q_table[state])]

    def update(self, state, action, reward, next_state, next_action):
        if state not in self.q_table:
            self.q_table[state] = np.zeros(len(self.actions))
        if next_state not in self.q_table:
            self.q_table[next_state] = np.zeros(len(self.actions))
        
        target = reward + self.gamma * self.q_table[next_state][self.actions.index(next_action)]
        self.q_table[state][self.actions.index(action)] += self.alpha * (target - self.q_table[state][self.actions.index(action)])

env = Environment()
actions = [0, 1]
sarsa_agent = SARSA(actions)

q_history = []

state = env.state
action = sarsa_agent.get_action(state)
for step in range(10):
    next_state, reward = env.step(action)
    next_action = sarsa_agent.get_action(next_state)
    sarsa_agent.update(state, action, reward, next_state, next_action)
    
    q_snapshot = {}
    for s in [0, 1]:
        if s in sarsa_agent.q_table:
            q_snapshot[s] = sarsa_agent.q_table[s].copy()
        else:
            q_snapshot[s] = np.zeros(len(actions))
    q_history.append(q_snapshot)
    
    state, action = next_state, next_action


    print(f"Step {step + 1}:")
    for s, q_values in sarsa_agent.q_table.items():
        print(f"  State {s}: {q_values}")
    
    state, action = next_state, next_action
print("")

print("Below is the Q_Table:")

print("")

print(f"{'Step':>4} | {'Q(0,0)':>6} {'Q(0,1)':>6} | {'Q(1,0)':>6} {'Q(1,1)':>6}")
print("-" * 38)
for i, q in enumerate(q_history, 1):
    print(f"{i:>4} | {q[0][0]:6.3f} {q[0][1]:6.3f} | {q[1][0]:6.3f} {q[1][1]:6.3f}")




Step 1:
  State 0: [0.1 0. ]
  State 1: [0. 0.]
Step 2:
  State 0: [0.1 0. ]
  State 1: [0.1 0. ]
Step 3:
  State 0: [0.1 0. ]
  State 1: [ 0.1   -0.091]
Step 4:
  State 0: [0.199 0.   ]
  State 1: [ 0.1   -0.091]
Step 5:
  State 0: [0.199 0.   ]
  State 1: [ 0.199 -0.091]
Step 6:
  State 0: [0.199 0.   ]
  State 1: [ 0.27091 -0.091  ]
Step 7:
  State 0: [0.199 0.   ]
  State 1: [ 0.27091 -0.16399]
Step 8:
  State 0: [0.3034819 0.       ]
  State 1: [ 0.27091 -0.16399]
Step 9:
  State 0: [0.3034819 0.       ]
  State 1: [ 0.3682009 -0.16399  ]
Step 10:
  State 0: [0.3034819 0.       ]
  State 1: [ 0.46451889 -0.16399   ]

Below is the Q_Table:

Step | Q(0,0) Q(0,1) | Q(1,0) Q(1,1)
--------------------------------------
   1 |  0.100  0.000 |  0.000  0.000
   2 |  0.100  0.000 |  0.100  0.000
   3 |  0.100  0.000 |  0.100 -0.091
   4 |  0.199  0.000 |  0.100 -0.091
   5 |  0.199  0.000 |  0.199 -0.091
   6 |  0.199  0.000 |  0.271 -0.091
   7 |  0.199  0.000 |  0.271 -0.164
   8 |  0.30